In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Dense, Embedding, Concatenate, StringLookup, Normalization, Flatten, 
                                     Lambda, Dropout, BatchNormalization, ELU, Reshape)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.data import TFRecordDataset
from customRatioLayerDefinition import RatioLayer, InverseLayer

# feature description
feature_description = {
    'tickers': tf.io.FixedLenFeature((288,), tf.float32),  
    'hourType': tf.io.FixedLenFeature((34,), tf.string),   
    'weekday': tf.io.FixedLenFeature((1,), tf.int64),
    'hour': tf.io.FixedLenFeature((1,), tf.int64),
    'month': tf.io.FixedLenFeature((1,), tf.int64),
    'target': tf.io.FixedLenFeature((1,), tf.int64),
}

def parse_example(serialized_example):
    example = tf.io.parse_single_example(serialized_example, feature_description)

    example['tickers'] = tf.reshape(example['tickers'], [288])  
    example['hourType'] = tf.reshape(example['hourType'], [34])  

    example['weekday'] = tf.cast(example['weekday'], tf.int32)
    example['hour'] = tf.cast(example['hour'], tf.int32)
    example['month'] = tf.cast(example['month'], tf.int32)
    example['target'] = tf.cast(example['target'], tf.int32)

    return example, example["target"]

# Load dataset
raw_dataset = TFRecordDataset("dataset.tfrecord").map(parse_example)
data_size = sum(1 for _ in raw_dataset)
print(f"✅ Total dataset size: {data_size}")

# Train/Val/Test split
train_size = int(0.7 * data_size)
val_size = int(0.15 * data_size)
test_size = data_size - (train_size + val_size)

dataset = TFRecordDataset("dataset.tfrecord").map(parse_example)
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size)

batch_size = 64
steps_per_epoch = train_size // batch_size

train_dataset = train_dataset.batch(batch_size).cache().prefetch(1)
val_dataset = val_dataset.batch(batch_size).cache().prefetch(1)
test_dataset = test_dataset.batch(batch_size).cache().prefetch(1)

print(f" Train: {train_size}, Validation: {val_size}, Test: {test_size}")

hourType_vocab = set()
for raw_example in raw_dataset.take(10000):  
    hourType_vocab.update(raw_example[0]['hourType'].numpy().astype(str))
hourType_vocab = sorted(hourType_vocab)

hourType_lookup = StringLookup(
    vocabulary=hourType_vocab, output_mode="int", num_oov_indices=1, mask_token=None
)

tickers_input = Input(shape=(288,), dtype=tf.float32, name="tickers")  
hourType_input = Input(shape=(34,), dtype=tf.string, name="hourType")  
weekday_input = Input(shape=(), dtype=tf.int32, name="weekday")
hour_input = Input(shape=(), dtype=tf.int32, name="hour")
month_input = Input(shape=(), dtype=tf.int32, name="month")

# RatioLayer & InverseLayer 
ratio_output = RatioLayer()(tickers_input)
inverse_output = InverseLayer()(tickers_input)

normalizer = Normalization()
normalizer.adapt(train_dataset.map(lambda x, y: x["tickers"]))
norm_output = normalizer(tickers_input)

hourType_indices = hourType_lookup(hourType_input)
hourType_embed = Embedding(input_dim=len(hourType_vocab) + 2, output_dim=16, mask_zero=True)(hourType_indices)

weekday_embed = Embedding(input_dim=7, output_dim=12)(weekday_input)
hour_embed = Embedding(input_dim=24, output_dim=12)(hour_input)
month_embed = Embedding(input_dim=12, output_dim=12)(month_input)

hourType_embed = Flatten()(hourType_embed)  
weekday_embed = Flatten()(weekday_embed)  
hour_embed = Flatten()(hour_embed) 
month_embed = Flatten()(month_embed)  

merged_features = Concatenate(axis=-1)([
    norm_output,        
    ratio_output,       
    inverse_output,    
    hourType_embed,    
    weekday_embed,      
    hour_embed,        
    month_embed    
])

merged_features = Flatten()(merged_features)

# Model Architecture
x = Dense(1024, kernel_regularizer=l2(0.0001))(merged_features)
x = BatchNormalization()(x)
x = ELU()(x)
x = Dropout(0.2)(x)

x = Dense(512, kernel_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = ELU()(x)
x = Dropout(0.2)(x)

x = Dense(256, kernel_regularizer=l2(0.0001))(x)
x = BatchNormalization()(x)
x = ELU()(x)
x = Dropout(0.2)(x)

output = Dense(26, activation="softmax")(x)

model = Model(inputs=[tickers_input, hourType_input, weekday_input, hour_input, month_input], outputs=output)

# Learning Rate
lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=0.001, first_decay_steps=2000, t_mul=2.0, m_mul=0.8
)

# Compile Model
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\n Model Training...\n")
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    callbacks=[early_stopping]
)
print("\n Model Training Completed!\n")

# Evaluate Model
print("\n Model Evaluation...\n")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"\n Model Evaluation Completed!")
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# Save Model
model.save("mySavedModel.keras")
print("\n Model Saved Successfully as 'mySavedModel.keras'!\n")


2025-03-04 22:42:45.217736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741146165.230017   16866 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741146165.233408   16866 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-04 22:42:45.248405: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1741146171.518249   16866 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 55

✅ Total dataset size: 29588
 Train: 20711, Validation: 4438, Test: 4439


2025-03-04 22:42:58.194846: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence



 Model Training...

Epoch 1/20


/home/akhil_tom/miniconda3/envs/AppliedML_tfcuda/lib/python3.12/site-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'flatten' (of type Flatten) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


...

2025-03-04 22:44:01.518943: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4670084778613932285
2025-03-04 22:44:01.519002: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 529670619456776559
2025-03-04 22:44:01.519014: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2122689796032431775
2025-03-04 22:44:01.549458: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8616470878313560579
2025-03-04 22:44:01.549531: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14847529329621378695
2025-03-04 22:44:01.549551: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2642682714144639520
2025-03-04 22:44:01.549560: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv ite

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4103 - loss: 2.1549 - val_accuracy: 0.2729 - val_loss: 2.3146


2025-03-04 22:44:02.108230: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4670084778613932285
2025-03-04 22:44:02.108299: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14876655026548204741
2025-03-04 22:44:02.108314: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10743761814813009321
2025-03-04 22:44:02.108323: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11035436740309923193
2025-03-04 22:44:02.108330: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14084602296768849985
2025-03-04 22:44:02.108337: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 529670619456776559
2025-03-04 22:44:02.108346: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv 


 Model Training Completed!


 Model Evaluation...

70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2790 - loss: 2.1281

 Model Evaluation Completed!
Test Loss: 2.1117, Test Accuracy: 0.2726

 Model Saved Successfully as 'mySavedModel.keras'!



2025-03-04 22:44:03.450076: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4670084778613932285
2025-03-04 22:44:03.450149: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14876655026548204741
2025-03-04 22:44:03.450166: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11035436740309923193
2025-03-04 22:44:03.450176: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10743761814813009321
2025-03-04 22:44:03.450183: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14084602296768849985
2025-03-04 22:44:03.450190: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 529670619456776559
2025-03-04 22:44:03.450199: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv 